# Link Prediction
## "People you May Know"

- Definition of Link Prediction
- Perform link prediction on dataset
   - Jaccard coefficient
   - Preferential Attachment


In [2]:
import networkx as nx
import matplotlib.pyplot as plt # for plotting graphs
%matplotlib inline

In [3]:
GA = nx.read_gexf('../data/ga_graph.gexf')

## Link Prediction

The idea of link prediction was first proposed by [Liben-Nowell and Kleinberg in 2004](https://www.cs.cornell.edu/home/kleinber/link-pred.pdf) as the following question:

> "Given a snapshot of a social network, can we infer which new interactions among its members are likely to occur in the near future?"

![](https://content.linkedin.com/content/dam/engineering/en-us/blog/migrated/link_prediction.jpg)

It's an inticing idea and has led to many interesting developments in the network literature. For our example, the question could be rephrased as:

> "Given a snapshot of the Grey's Anatomy relationship network, can we infer which new relationships are likely to occur in the near future?"

Sounds awesome, but how does it work? 

## Jaccard Coefficient

The most popular measures for link prediction analyze the “proximity” of nodes in a network. One way to measure proximity is to see what proportion of neighbors a pair of nodes share (more common connections suggests there's a higher chance that the two unmatched nodes will connect).  This can be capture succintly with the Jaccard index.  

![](https://upload.wikimedia.org/math/0/a/0/0a0633ce67c9130d890078a8d67f0474.png)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Intersection_of_sets_A_and_B.svg/128px-Intersection_of_sets_A_and_B.svg.png)

In the context of a network, we're comparing sets of neighbors:

$$ Jaccard = \frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u) \cup \Gamma(v)|} $$

where $\Gamma(u)$ denotes the set of neighbors of $u$.

In [14]:
preds_jc = nx.jaccard_coefficient(GA)

In [15]:
pred_jc_dict = {}
for u, v, p in preds_jc:
    pred_jc_dict[(u,v)] = p

In [16]:
sorted(pred_jc_dict.items(), key=lambda x:x[1], reverse=True)[:10]

[(('steve', 'finn'), 1.0),
 (('tucker', 'ben'), 1.0),
 (('mrs. seabury', 'kepner'), 1.0),
 (('colin', 'preston'), 1.0),
 (('denny', 'hank'), 1.0),
 (('lexi', 'addison'), 0.5),
 (('olivia', 'izzie'), 0.5),
 (('preston', 'owen'), 0.5),
 (('derek', 'finn'), 0.5),
 (('torres', 'olivia'), 0.5)]

In [43]:
extra_attrs = {'finn':('Finn Dandridge','M','S'),
 'olivia':('Olivia Harper','F','S'),
 'steve':('Steve Murphy','M','S'),
 'torres':('Callie Torres','F','B'),
 'colin':('Colin Marlow','M','S'),
 'grey':('Meredith Grey','F','S'),
 'mrs. seabury':('Dana Seabury','F','S'),
 'altman':('Teddy Altman','F','S'),
 'tucker':('Tucker Jones','M','S'),
 'ben':('Ben Warren','M','S'),
 "o'malley":("George O'Malley",'M','S'),
 'thatch grey':('Thatcher Grey','M','S'),
 'susan grey':('Susan Grey','F','S'),
 'derek':('Derek Shepherd','M','S'),
 'chief':('Richard Webber','M','S'),
 'addison':('Addison Montgomery','F','S'),
 'karev':('Alex Karev','M','S'),
 'hank':('Hank','M','S'),
 'lexi':('Lexie Grey','F','S'),
 'adele':('Adele Webber','F','S'),
 'owen':('Owen Hunt','M','S'),
 'sloan':('Mark Sloan','M','S'),
 'arizona':('Arizona Robbins','F','G'),
 'izzie':('Izzie Stevens','F','S'),
 'preston':('Preston Burke','M','S'),
 'kepner':('April Kepner','M','S'),
 'bailey':('Miranda Bailey','F','S'),
 'ellis grey':('Ellis Grey','F','S'),
 'denny':('Denny Duquette','M','S'),
 'yang':('Cristina Yang','F','S'),
 'nancy':('Nancy Shepherd','F','S'),
 'avery':('Jackson Avery','M','S')}

In [45]:
for i in GA.nodes():
    GA.node[i]["full_name"]   = extra_attrs[i][0]
    GA.node[i]["gender"]      = extra_attrs[i][1]
    GA.node[i]["orientation"] = extra_attrs[i][2]

In [47]:
GA.node['grey']

{'full_name': 'Meredith Grey',
 'gender': 'F',
 'label': 'grey',
 'orientation': 'S'}

## Preferential Attachment

Though shared connections is an intuitive way to predict future connections, there other variations may better reflect the underlying social mechanisms of linking.

The preferential attachement methods mirrors the “rich get richer” mantra by supposing that nodes with more connections will be the ones to be more likely to get future connections. In the context of social networks, the ones that have more social connectivity (the “hubs”) receive more new connections than the poorly connected. 

![](http://www.frumforum.com/wp-content/uploads/2011/10/rich1.jpg)
 
 > Preferential attachment has received considerable attention as a model of the growth of networks. The basic premise is that the probability that a new edge involves node x is proportional to |Γ(x)|, the current number of neighbors of x. Newman and Barabasi et al. have further proposed, on the basis of empirical evidence, that the probability of co authorship of x and y is correlated with the product of the number of collaborators of x and y. This corresponds to the measure score(x, y) := |Γ(x)| · |Γ(y)|.

Preferential attachment score of $u$ and $v$ is defined as:

$|\Gamma(u)| |\Gamma(v)|$

where $\Gamma(u)$ denotes the set of neighbors of $u$.

In [18]:
preds_pa = nx.preferential_attachment(GA)

In [19]:
pred_pa_dict = {}
for u, v, p in preds_pa:
    pred_pa_dict[(u,v)] = p

In [21]:
sorted(pred_pa_dict.items(), key=lambda x:x[1], reverse=True)[:10]

[(('karev', 'sloan'), 35),
 (('karev', "o'malley"), 28),
 (('karev', 'grey'), 28),
 (('karev', 'yang'), 21),
 (('izzie', 'sloan'), 20),
 (('sloan', "o'malley"), 20),
 (('grey', 'sloan'), 20),
 (('torres', 'izzie'), 16),
 (('torres', 'grey'), 16),
 (('grey', 'izzie'), 16)]